## Лабораторна робота `№ 2`: перевірка статистичної гіпотези про вигляд розподілу і гіпотези однорідності 

In [1]:
from scipy import stats
import numpy as np
from scipy.special import kolmogi

In [2]:
gamma = 0.05

## Завдання `1`:  Критерій Колмогорова

In [3]:
def exponen_ds(x,alpha): # функція екпоненціального розподілу
    return 1-np.exp(-alpha*x)

In [31]:
def kolmogorov_experiment(sample, alpha):
    sample = np.sort(sample) # варіаційний ряд 
    n = len(sample)
    k = np.array(range(1,n+1))
    
    d_plus = exponen_ds(sample,alpha) - (k-1)/n # знаходимо Dnm+
    d_minus = k/n - exponen_ds(sample,alpha) # знаходимо Dnm-
    
    D_n = np.maximum(d_plus,d_minus).max()
    krit = kolmogi(gamma)/np.sqrt(n) # критичне значення
    if D_n > krit:
        return ['Cуперечить','D_n='+str(D_n),'krit='+str(krit)]
    else:
        return ['Не суперечить','D_n='+str(D_n),'krit='+str(krit)]

```
а) Ho: X_i~F(u,1), коли насправді X_i~F(u,1)
```

In [32]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size = n)# вхідне alpha=1 
    print('n = ',i,"Результат:",kolmogorov_experiment(X,alpha = 1)) #перевірка alpha=1

n =  1000 Результат: ['Не суперечить', 'D_n=0.017022088365959753', 'krit=0.042946849874347756']
n =  10000 Результат: ['Не суперечить', 'D_n=0.007000510452766173', 'krit=0.013580986393225507']
n =  100000 Результат: ['Не суперечить', 'D_n=0.0024906266768829344', 'krit=0.004294684987434775']


```
б) Ho: X_i~F(u,1), коли насправді X_i~F(u,1.2)
```

In [35]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size = n)# вхідне alpha=1
    print('n = ',i,"Результат:",kolmogorov_experiment(X,alpha = 1.2)) #перевірка alpha=1.2

n =  1000 Результат: ['Cуперечить', 'D_n=0.06969543031942305', 'krit=0.042946849874347756']
n =  10000 Результат: ['Cуперечить', 'D_n=0.07332724668926438', 'krit=0.013580986393225507']
n =  100000 Результат: ['Cуперечить', 'D_n=0.06983857866146748', 'krit=0.004294684987434775']


## Завдання `2`: Критерій Xі^2

In [4]:
from scipy.special import chdtri

In [5]:
def expon_probab(a,b,alpha): # теоретична ймовірність
    return np.exp(-alpha*a) - np.exp(-alpha*b)

In [6]:
def chi_experiment(X,alpha):
    n = len(X)
    r = int(20*n/1000)
    
    nu = []
    teori_probab = []
    split_list = np.linspace(0,10,r) #розбиття
    
    for i in range(1,len(split_list)):
        a = X[X > split_list[i-1]]
        b =  X[X < split_list[i]]
        nu.append(np.intersect1d(a,b).shape[0])
        teori_probab.append(expon_probab(split_list[i-1],split_list[i], alpha))   
    
    nu,teori_probab = np.array(nu),np.array(teori_probab)
    
    delta = np.sum(((nu - n*teori_probab)**2)/(n*teori_probab)) # рахуємо дельту
    krit = chdtri(r-1,gamma)# критичне значення
    if delta > krit:
        return ['Cуперечить','delta='+str(delta),'krit='+str(krit)]
    else:
        return ['Не суперечить','delta='+str(delta),'krit='+str(krit)]

```
а) Ho: X_i~F(u,1), коли насправді X_i~F(u,1)
```

In [18]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size = n) #вхідне альфа 1
    print('n = ',i,"Результат:",chi_experiment(X,1))#перевірка alpha=1

n =  1000 Результат: ['Не суперечить', 'delta=11.073709579655091', 'krit=30.143527205646155']
n =  10000 Результат: ['Не суперечить', 'delta=143.62870473184626', 'krit=232.91182176847585']
n =  100000 Результат: ['Не суперечить', 'delta=2032.1325562879938', 'krit=2104.128222359781']


```
б) Ho: X_i~F(u,1), коли насправді X_i~F(u,1.2)
```

In [19]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size = n)# вхідне alpha=1
    print('n = ',i,"Результат:",chi_experiment(X,1.2))#перевірка alpha=1.2

n =  1000 Результат: ['Cуперечить', 'delta=52.86215425094681', 'krit=30.143527205646155']
n =  10000 Результат: ['Cуперечить', 'delta=970.5218592473534', 'krit=232.91182176847585']
n =  100000 Результат: ['Cуперечить', 'delta=8732.178426763148', 'krit=2104.128222359781']


## Завдання `3`: Критерій пустих ящиків

In [20]:
def empty_boxes_experiment(distr,alpha):
    n  = len(distr)
    ro = 2
    r  = int(n/2)
    
    m = np.exp(-ro)# мат.очікування
    sigma2 = np.exp(-ro)*(1-(1+ro)*np.exp(-ro))# дисперсія
    
    z_gamma = stats.norm.ppf(0.5-gamma) 
    
    split_list = np.linspace(0,1,r) # розбиття
    
    Y = exponen_ds(distr,alpha)
    check = r*m +z_gamma*np.sqrt(r)*np.sqrt(sigma2)# критичне значення
    nu = 0
    for i in range(1,len(split_list)):
        a = Y[Y > split_list[i-1]]
        b = Y[Y < split_list[i]]
        if np.intersect1d(a,b).shape[0] ==0:
            nu+=1
    if nu > check:
        return ['Cуперечить','nu='+str(nu),'krit='+str(check)]
    else:
        return ['Не суперечить','nu='+str(nu),'krit='+str(check)]

```
а) Ho: X_i~F(u,1), коли насправді X_i~F(u,1)
```

In [14]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size = n)# вхідне alpha=1
    print('n = ',i,"Результат:",empty_boxes_experiment(X, 1))#перевірка alpha=1.2

n =  1000 Результат: ['Не суперечить', 'nu=64', 'krit=66.87096271929707']
n =  10000 Результат: ['Не суперечить', 'nu=667', 'krit=674.157096298399']
n =  100000 Результат: ['Не суперечить', 'nu=6743', 'krit=6758.797372840542']


```
б) Ho: X_i~F(u,1), коли насправді X_i~F(u,1.2)
```

In [21]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size = n)#вхідне alpha=1
    print('n = ',i,"Результат:",empty_boxes_experiment(X, 1.2))#перевірка alpha=1.2

n =  1000 Результат: ['Cуперечить', 'nu=68', 'krit=66.87096271929707']
n =  10000 Результат: ['Cуперечить', 'nu=757', 'krit=674.157096298399']
n =  100000 Результат: ['Cуперечить', 'nu=6815', 'krit=6758.797372840542']


## Завдання `4`: Критерій однорідності Смирнова 

In [24]:
#емпірична функція розподілу
def F(u,X):
    return X[X<u].shape[0]/len(X)

In [27]:
def smirnov_test(distr_1,distr_2):
    distr_1 = np.sort(distr_1)
    distr_2 = np.sort(distr_2)
    
    n = len(distr_1)
    m =len(distr_2)
    
    z_gamma = kolmogi(gamma)
    krit = z_gamma*np.sqrt(1/n + 1/m)# критичне значення
    
    d_plus =  max([k/m-F(distr_2[k-1],distr_1)      for k in range(1,m+1)])# знаходимо Dnm+
    d_minus = max([F(distr_2[k-1],distr_1)-(k-1)/m  for k in range(1,m+1)])# знаходимо Dnm-
    
    d = max([d_plus,d_minus])#D
    
    if d > krit:
        return ['Cуперечить','D='+str(d),'krit='+str(krit)]
    else:
        return ['Не суперечить','D='+str(d),'krit='+str(krit)]


-----------------------------
```
Ho: X_1 = (x_1_1,x_1_2,...x_1_n)~F(u,1)
    X_2 = (x_2_1,x_2_2,...x_2_n)~F(u,1)
Насправді X_1~F(u,1), X_2~F(u,1)
```

In [28]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size=n)# перший розподіл alpha=1
    Y = np.random.exponential(1,size=int(n/2))# другий розподіл alpha=1
    print('n = ',i,"Результат:",smirnov_test(X,Y))# перевірка на однорідність

n =  1000 Результат: ['Не суперечить', 'D=0.04200000000000001', 'krit=0.07438612600740337']
n =  10000 Результат: ['Не суперечить', 'D=0.0131', 'krit=0.023522958449968174']
n =  100000 Результат: ['Не суперечить', 'D=0.003129999999999966', 'krit=0.007438612600740337']


---------------------------
```
Ho: X_1 = (x_1_1,x_1_2,...x_1_n)~F(u,1)
    X_2 = (x_2_1,x_2_2,...x_2_n)~F(u,1)
Насправді X_1~F(u,1), X_2~F(u,1.2)
```

In [30]:
for i in [10**3,10**4,10**5]:
    n=i
    X = np.random.exponential(1,size=n)# перший розподіл alpha=1
    Y = np.random.exponential(1.2,size=int(n/2))# другий розподіл alpha=1.2
    print('n = ',i,"Результат:",smirnov_test(X,Y))# перевірка на однорідність

n =  1000 Результат: ['Cуперечить', 'D=0.12699999999999995', 'krit=0.07438612600740337']
n =  10000 Результат: ['Cуперечить', 'D=0.06690000000000002', 'krit=0.023522958449968174']
n =  100000 Результат: ['Cуперечить', 'D=0.07123000000000002', 'krit=0.007438612600740337']
